# General Practice Workforce

In [2]:
# this is a bit of a hack to get relative imports
# to work as if these notebooks were in a package
# change cwd to project root if 'notebooks' in PATH
from os import chdir
from pathlib import Path

if "notebooks" in str(Path.cwd()):
    chdir("..")


import src.constants as constants
from src.schemas import DataCatalog
from src.various_methods import (
    PlotCounter,
    )
import yaml

In [3]:
NOTEBOOK_ALIAS =  "WORKFORCE"
WORKFORCE_CATALOG_ENTRY_NAME = "General Practice workforce"
plot_counter = PlotCounter(name=NOTEBOOK_ALIAS)
# load data catalog
catalog = DataCatalog.load_from_yaml()


# Mapping Clinical staff

### calculating % mix of non-GP

- Advanced Nurse Practitioners
- Nurses
- Other Direct Patient Care


In [4]:
workforce_entry =  catalog.get_catalog_entry_by_name("General Practice workforce")
workforce_df = workforce_entry.load()

In [17]:
# select only SNEE
non_gp_staff_mix = workforce_df.loc[
    (workforce_df["SUB_ICB_CODE"].isin(constants.SUB_ICB_CODES.keys()))]

# set staff group to 'Advanced Nurse Practitioners' if 'STAFF_ROLE' is 'Advanced Nurse Practitioners' in clinical_df
non_gp_staff_mix.loc[non_gp_staff_mix['STAFF_ROLE'] == 'Advanced Nurse Practitioners', 'STAFF_GROUP'] = 'Advanced Nurse Practitioners'

# fix typo in 'Admin/Non-clinical' in 'STAFF_GROUP'
non_gp_staff_mix = non_gp_staff_mix.replace({'Admin/Non-clinical':'Admin/Non-Clinical'})

# groupby sum to get FTE
staff_fte =  non_gp_staff_mix.groupby(['SUB_ICB_CODE', 'STAFF_GROUP'])[['FTE']].sum().reset_index()

In [18]:
# drop 'GP' and 'Admin/Non-Clinical' staff groups
staff_fte = staff_fte.loc[~staff_fte['STAFF_GROUP'].isin(['GP', 'Admin/Non-Clinical'])]

# sum of FTE for each SUB_ICB_CODE
staff_fte_sum = staff_fte.groupby('SUB_ICB_CODE')['FTE'].sum()

In [23]:
non_gp_clinical_staff_mix = staff_fte.merge(staff_fte_sum, on='SUB_ICB_CODE', suffixes=('', '_sum')).assign(fte_percent=lambda x: x['FTE'] / x['FTE_sum'] * 100).drop(columns=['FTE_sum','FTE']).round(3)



In [24]:
non_gp_clinical_staff_mix

,SUB_ICB_CODE,STAFF_GROUP,fte_percent
0,06L,Advanced Nurse Practitioners,13.514
1,06L,Direct Patient Care,59.338
2,06L,Nurses,27.148
3,06T,Advanced Nurse Practitioners,20.494
4,06T,Direct Patient Care,44.154
5,06T,Nurses,35.352
6,07K,Advanced Nurse Practitioners,8.599
7,07K,Direct Patient Care,59.185
8,07K,Nurses,32.216


In [9]:
# if the staf

non_gp_staff_mix.replace({'Admin/Non-clinical':'Admin/Non-Clinical'}).groupby(['SUB_ICB_CODE', 'STAFF_GROUP'])[['FTE']].sum()

FTE
SUB_ICB_CODE STAFF_GROUP                    
06L          Admin/Non-Clinical   569.526796
             Direct Patient Care  196.760585
             GP                   223.145140
             Nurses               134.834859
06T          Admin/Non-Clinical   480.339551
             Direct Patient Care  109.227660
             GP                   176.349687
             Nurses               138.148452
07K          Admin/Non-Clinical   365.215680
             Direct Patient Care  123.520000
             GP                   175.831904
             Nurses                85.181021

In [6]:
"""
This code loads a data catalog, retrieves a specific catalog entry, and performs data manipulation and aggregation on the loaded data.
The resulting aggregated data is then saved as a YAML file.

Functions:
    - load_from_yaml(): Loads the data catalog from a YAML file.
    - get_catalog_entry_by_name(name): Retrieves a catalog entry by its name.
    - load(): Loads the data associated with a catalog entry.
"""


workforce_entry =  catalog.get_catalog_entry_by_name("General Practice workforce")
workforce_df = workforce_entry.load()

non_gp_staff_mix = workforce_df.loc[
    (~workforce_df["STAFF_GROUP"].isin(['Admin/Non-clinical','Admin/Non-Clinical']))&
    (workforce_df["SUB_ICB_CODE"].isin(constants.SUB_ICB_CODES.keys()))]

output_df = non_gp_staff_mix.groupby(['SUB_ICB_CODE','STAFF_GROUP'])[['FTE']].sum(numeric_only=True).reset_index()

output_dict ={}
# Group the DataFrame by 'SUB_ICB_CODE'
grouped = output_df.groupby('SUB_ICB_CODE')

# Loop through each group and create the required nested dictionary
for name, group in grouped:
    output_dict[name] = {row['STAFF_GROUP']: row['FTE'] for _, row in group.iterrows()}

with open("outputs/staff_fte.yaml", 'w') as yaml_file:
    yaml.dump(output_dict, yaml_file)

In [8]:
from src.simulation_schemas import DidNotAttendRatesByArea

did_not_attend_rates = DidNotAttendRatesByArea.read_yaml("outputs/assumptions/dna_appointments.yaml")

In [9]:
from src.various_methods import get_workingdays, is_working_day

In [10]:
#population scenarios
# opulation_scenarios:PopulationScenarios = PopulationScenarios.read_yaml("../outputs/population_projections.yaml")
# staffing levels
from src.simulation_schemas import ClinicalStaffFTEByArea, MonthlyAppointmentForecast, StaffPropensityByArea,AreaAppointmentTimeDistributions


staff_fte = ClinicalStaffFTEByArea.read_yaml("outputs/staff_fte.yaml")
# did not attend
did_not_attend_rates = DidNotAttendRatesByArea.read_yaml("outputs/assumptions/dna_appointments.yaml")
staff_type_propensity = StaffPropensityByArea.read_yaml("outputs/assumptions/staff_propensity.yaml")
appointment_durations = AreaAppointmentTimeDistributions.read_yaml("outputs/assumptions/appointment_duration.yaml")

In [11]:
from src.simulation_schemas import DeliveryPropensityByArea


appointment_type = DeliveryPropensityByArea.read_yaml("outputs/assumptions/appointment_modes.yaml")

In [12]:
import pandas as pd
df = pd.read_csv("outputs/simulation_outputs.csv")

In [18]:
snee_workforce_df = workforce_df.loc[
    (workforce_df["SUB_ICB_CODE"].isin(constants.SUB_ICB_CODES.keys()))]

In [24]:
snee_workforce_df.sort_values('DETAILED_STAFF_ROLE')

,YEAR,Month,COMM_REGION_CODE,COMM_REGION_NAME,ICB_CODE,ICB_NAME,SUB_ICB_CODE,SUB_ICB_NAME,DATA_SOURCE,UNIQUE_IDENTIFIER,STAFF_GROUP,DETAILED_STAFF_ROLE,STAFF_ROLE,COUNTRY_QUALIFICATION_AREA,COUNTRY_QUALIFICATION_GROUP,AGE_BAND,AGE_YEARS,GENDER,FTE
165995,2024,4,Y61,East of England,QJG,NHS Suffolk and North East Essex ICB,07K,NHS Suffolk and North East Essex ICB - 07K Wes...,Provided,129079,Nurses,Advanced Nurse Practitioner,Advanced Nurse Practitioners,Not Applicable,Not Applicable,45-49,47.0,Female,0.533333
86979,2024,4,Y61,East of England,QJG,NHS Suffolk and North East Essex ICB,06L,NHS Suffolk and North East Essex ICB - 06L Ips...,Provided,83086,Nurses,Advanced Nurse Practitioner,Advanced Nurse Practitioners,Not Applicable,Not Applicable,65 and over,65.0,Female,0.640000
87953,2024,4,Y61,East of England,QJG,NHS Suffolk and North East Essex ICB,06L,NHS Suffolk and North East Essex ICB - 06L Ips...,Provided,71719,Nurses,Advanced Nurse Practitioner,Advanced Nurse Practitioners,Not Applicable,Not Applicable,50-54,50.0,Female,0.680000
128993,2024,4,Y61,East of England,QJG,NHS Suffolk and North East Essex ICB,06T,NHS Suffolk and North East Essex ICB - 06T Nor...,Provided,127516,Nurses,Advanced Nurse Practitioner,Advanced Nurse Practitioners,Not Applicable,Not Applicable,25-29,25.0,Female,0.600000
32891,2024,4,Y61,East of England,QJG,NHS Suffolk and North East Essex ICB,06L,NHS Suffolk and North East Essex ICB - 06L Ips...,Provided,132009,Nurses,Advanced Nurse Practitioner,Advanced Nurse Practitioners,Not Applicable,Not Applicable,45-49,46.0,Female,0.613333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176226,2024,4,Y61,East of England,QJG,NHS Suffolk and North East Essex ICB,06L,NHS Suffolk and North East Essex ICB - 06L Ips...,Provided,3968,Nurses,Trainee Nurse,Trainee Nurses,Not Applicable,Not Applicable,35-39,36.0,Female,1.000000
178964,2024,4,Y61,East of England,QJG,NHS Suffolk and North East Essex ICB,06L,NHS Suffolk and North East Essex ICB - 06L Ips...,Provided,177490,Direct Patient Care,Trainee Nursing Associate,Trainee Nursing Associates,Not Applicable,Not Applicable,Under 25,23.0,Female,0.920000
133366,2024,4,Y61,East of England,QJG,NHS Suffolk and North East Essex ICB,07K,NHS Suffolk and North East Essex ICB - 07K Wes...,Provided,206208,Direct Patient Care,Trainee Nursing Associate,Trainee Nursing Associates,Not Applicable,Not Applicable,35-39,38.0,Female,0.986667
126258,2024,4,Y61,East of England,QJG,NHS Suffolk and North East Essex ICB,06L,NHS Suffolk and North East Essex ICB - 06L Ips...,Provided,179100,Direct Patient Care,Trainee Nursing Associate,Trainee Nursing Associates,Not Applicable,Not Applicable,25-29,25.0,Female,1.000000


In [22]:
sorted(snee_workforce_df['DETAILED_STAFF_ROLE'].unique().tolist())

['Advanced Nurse Practitioner',
 'Advanced Paramedic Practitioner',
 'Advanced Pharmacist Practitioner',
 'Apprentice',
 'Apprentice - Healthcare Assistant',
 'Apprentice - Other',
 'Dispenser',
 'Estates and Ancillary',
 'Extended Role Practice Nurse',
 'GP in Training Grade ST1',
 'GP in Training Grade ST2',
 'GP in Training Grade ST3',
 'General Practice Assistant',
 'Health Support Worker',
 'Healthcare Assistant',
 'Locum - Covering Sickness/Maternity/Paternity',
 'Locum - Covering Vacancy',
 'Locum - other',
 'Management Partner',
 'Manager',
 'Medical Secretary',
 'Nurse Dispenser',
 'Nurse Specialist',
 'Nursing Associate',
 'Nursing Partner',
 'Other',
 'Paramedic',
 'Partner/Provider',
 'Pharmacist',
 'Pharmacy Technician',
 'Phlebotomist',
 'Physician Associate',
 'Physiotherapist',
 'Practice Nurse',
 'Receptionist',
 'Retainer',
 'Salaried By Other',
 'Salaried By Practice',
 'Senior Partner',
 'Telephonist',
 'Trainee Nurse',
 'Trainee Nursing Associate']

In [ ]:
'Nurses', Admin/Non-clinical,

In [ ]:
['Advanced Nurse Practitioner':'Nurses',
 'Advanced Paramedic Practitioner',
 'Advanced Pharmacist Practitioner',
 'Apprentice',
 'Apprentice - Healthcare Assistant',
 'Apprentice - Other',
 'Dispenser',
 'Estates and Ancillary',
 'Extended Role Practice Nurse',
 'GP in Training Grade ST1',
 'GP in Training Grade ST2',
 'GP in Training Grade ST3',
 'General Practice Assistant',
 'Health Support Worker',
 'Healthcare Assistant',
 'Locum - Covering Sickness/Maternity/Paternity',
 'Locum - Covering Vacancy',
 'Locum - other',
 'Management Partner',
 'Manager',
 'Medical Secretary',
 'Nurse Dispenser',
 'Nurse Specialist',
 'Nursing Associate',
 'Nursing Partner',
 'Other',
 'Paramedic',
 'Partner/Provider',
 'Pharmacist',
 'Pharmacy Technician',
 'Phlebotomist',
 'Physician Associate',
 'Physiotherapist',
 'Practice Nurse',
 'Receptionist',
 'Retainer',
 'Salaried By Other',
 'Salaried By Practice',
 'Senior Partner',
 'Telephonist',
 'Trainee Nurse',
 'Trainee Nursing Associate']